In [1]:
import pandas as pd
from selenium import webdriver
import time
import re
from bs4 import BeautifulSoup

In [8]:
df = pd.read_csv('대학병원(종합).csv',engine='python',encoding='UTF-8')

In [10]:
from konlpy.tag import Twitter
from konlpy.utils import pprint

In [11]:
twt=Twitter()

In [14]:
cc=df.loc[:,"증상"]

In [15]:
# 동사
형태소=[]
for i,k in enumerate(cc):
    tok=twt.pos(k,norm=True)
    nouns=[n for n, tag in tok if (tag=='Verb')|(tag=='Noun')]
#     pprint(nouns)
    형태소.append(nouns)
df['형태소']=형태소

In [134]:
df.tail()

,부위,질환명(국문),질환명(영문),증상,치료,진료과,형태소
2158,NaN,구강 작열감 증후군,Burning mouth syndrome,"주로 혀의 끝 부분에 화끈거림, 타는 듯한 느낌이 나타난다. 쑤시는 것 같기도 하고...","엽산이나 비타민 B12, 철분 결핍 시 보충해 준다. \n- 염증이 있을 경우에는 ...",NaN,"[주로, 혀, 끝, 부분, 화끈, 거림, 타는, 듯, 느낌, 나타난, 쑤시, 것, ..."
2159,NaN,활액낭염(윤활낭염),Bursitis,"주로 나타나는 증상은 통증이며, 잘 나타나는 부위는 무릎, 팔꿈치, 어깨, 손목 등...",초기에는 며칠 정도 휴식을 취하면 활액낭 내의 액체가 흡수되면서 가라앉는다. \n-...,NaN,"[주로, 나타나는, 증상, 통증, 잘, 나타나는, 부위, 무릎, 팔꿈치, 어깨, 손..."
2160,NaN,소아 비만,Child obesity,일반적으로 비만은 상체 비만과 하체 비만으로 나뉜다. 상체 비만에 속하는 복부 비만...,식사 요법과 운동 요법으로 비만을 조절하는 것이 가장 중요하다. \n\n- 식사 요...,NaN,"[일반, 비만, 상체, 비만, 하체, 비만, 나뉜, 상체, 비만, 속하는, 복부, ..."
2161,NaN,용혈 빈혈,Hemolytic anemia,용혈 빈혈은 일반적인 빈혈 증상과 함께 용혈로 인한 특이 소견들이 함께 나타난다. ...,\n치료 목표는 빈혈의 증상을 완화하고 적혈구 파괴에 의한 신장 기능의 저하를 예방...,NaN,"[용혈, 빈혈, 일반, 빈혈, 증상, 용혈, 인한, 특이, 소견, 나타난, 일반, ..."
2162,NaN,자가 면역 용혈 빈혈,Autoimmune hemolytic anemia,"일반적인 무증상에서 치명적인 임상 증상까지 다양하다. 우선 만성 피로, 간과 비장의...",치료는 대개 증상 완화 위주로 이루어진다. \n- 치명적인 자가 면역 용혈 빈혈인 ...,NaN,"[일반, 무증상, 치명, 임, 증상, 우선, 만성, 피로, 간과, 비장, 비대, 운..."


In [18]:
len(nouns)

119

In [ ]:
table2 = pd.DataFrame(nouns,columns=['형태소'])
# table2

In [92]:
p = re.compile("[^0-9]")

In [169]:
# 아래 '~~' 에는 크롬드라이버 chromedriver.exe의 경로를 적어줄 것.
# 예시 driver = webdriver.Chrome('C:/Users/chromedriver.exe')
driver = webdriver.Chrome('~~')
# 주소 접속
driver.get('https://ko.dict.naver.com/#/main')

In [170]:
# 네이버 국어사전 접속시 처음 뜨는 팝업창 끄기
driver.find_element_by_css_selector('button.btn_nomore.my_notice_btn_nomore').click()

In [124]:
# 유의어/반의어 구별X
유의어반의어 = [];
for i in nouns:
    # ac_input 칸 clear. 검색 단어가 여전히 남아있기 때문에 지워줘야 함.
    driver.find_element_by_id('ac_input').clear()
    # 검색칸에 단어를 입력함.
    driver.find_element_by_id('ac_input').send_keys(i)
    
    # 네이버 국어사전에 너무 많은 패킷을 보내게 되면 IP를 일시적으로 차단할 수도 있음.
    # 이를 방지하기 위해 2초 정도 코드 실행을 쉼.
    driver.implicitly_wait(2)
    
    driver.find_element_by_class_name('btn_search').click()
    a = []
    try:
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        # 크롬 개발자 모드(F12)를 통해 현재 페이지의 소스코드를 확인해봄.
        # 유의어/반의어는 dd class="cont" 의 a태그에 존재
        notices = soup.select('dd.cont a')
        for n in notices:
#             print("".join(p.findall(n.text.strip()))
            a.append("".join(p.findall(n.text.strip())))
    except:
        print("유의어/반의어가 없음")
    유의어반의어.append(a)
table2['유의어반의어'] = 유의어반의어
table2

,형태소,유의어반의어
0,일반,"[총체, 보편, 보통, 특수]"
1,무증상,[]
2,치명,[순교]
3,임,[]
4,증상,"[증후, 병세, 증세]"
5,우선,"[그럭저럭, 되는대로, 위선]"
6,만성,"[버릇, 습관, 급성]"
7,피로,"[주럽, 주접, 피곤]"
8,간과,"[무기, 전쟁]"
9,비장,"[비위, 지라]"


In [176]:
table2 = pd.DataFrame(nouns[:10],columns=['형태소'])
table2

,형태소
0,일반
1,무증상
2,치명
3,임
4,증상
5,우선
6,만성
7,피로
8,간과
9,비장


In [202]:
# 유의어 반의어 구별
유의어 = []
반의어 = []
for i in nouns[:10]:
    driver.implicitly_wait(1)
    driver.find_element_by_id('ac_input').clear()
    driver.find_element_by_id('ac_input').send_keys(i)
    driver.find_element_by_class_name('btn_search').click()
    a = []; b=[]
    body1 = []; body2 = []
    try:
        # 유의어/반의어를 구분하기 위해 단순히 단어만 끌어오는 것이 아니라
        # 그 단어의 상위 집합으로 비교하는 작업 필요.
        body1 = driver.find_elements_by_css_selector('dt.tit')
        body2 = driver.find_elements_by_css_selector('dd.cont')
    except:
        print("유의어/반의어가 없음")
    if body1 != [] :
        # 상위 집합들을 순회하며 유의어/반의어 구분해서 저장
        for i in range(len(body1)):
            if body1[i].text == '유의어':
                a.extend("".join(p.findall(body2[i].text)).split())
            else:
                b.extend("".join(p.findall(body2[i].text)).split())
    유의어.append(a)
    반의어.append(b)
table2['유의어'] = 유의어
table2['반의어'] = 반의어

In [203]:
table2

,형태소,유의어,반의어
0,일반,"[총체, 보편, 보통]",[특수]
1,무증상,[],[]
2,치명,[순교],[]
3,임,[],[]
4,증상,"[증후, 병세, 증세]",[]
5,우선,"[그럭저럭, 되는대로, 위선]",[]
6,만성,"[버릇, 습관]",[급성]
7,피로,"[주럽, 주접, 피곤]",[]
8,간과,"[무기, 전쟁]",[]
9,비장,"[비위, 지라]",[]


In [204]:
table2['유의어'][0][1]

'보편'